In [33]:
p1 = "/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained_reprojected/scans/scene0000_00/create_labels_from_pretrained_reprojected"
p2 = "/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained/scans/scene0000_00/create_labels_from_pretrained"
p3 = "/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt"

from pathlib import Path

p1 = [str(s) for s in Path(p1).rglob('*.png')]
p1.sort(
    key=lambda x: int(x.split("/")[-1][:-4])
)
p2 = [str(s) for s in Path(p2).rglob('*.png')]
p2.sort(
    key=lambda x: int(x.split("/")[-1][:-4])
)
p3 = [str(s) for s in Path(p3).rglob('*.png')]
p3.sort(
    key=lambda x: int(x.split("/")[-1][:-4])
)

images = [ s.replace("/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained_reprojected/scans/scene0000_00/create_labels_from_pretrained_reprojected", "/home/jonfrey/Datasets/scannet/scans/scene0000_00/color").replace('png', 'jpg') for s in p1 ]
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")
from utils_asl import LabelLoaderAuto
from visu import Visualizer
from PIL import Image
visu = Visualizer("/home/jonfrey/Documents/master_thesis/Report/images/reprojected_pretrained", 
                  logger=None, 
                  epoch=0, store=True, num_classes=41)

lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet")

In [25]:
replot_sequence = False
if replot_sequence:
    for k in range(len(p1)):
        l = lla.get(p1[k])[0]
        img = np.array( Image.open( images[k]) )
        visu.plot_detectron(img, l, alpha=0.8 ,tag="reprojected", jupyter=False, text_off=True,epoch=k*10)


In [62]:
import copy
import numpy as np

class AccMonitor():
    def __init__(self):
        self.metrics = {}
        self.metric_dict = {'correct': 0,
                            'correct_valid': 0,
                            'correct_valid_both': 0,
                            'total': 0, 
                            'total_valid':0,
                            'total_valid_both':0,
                            'images': 0}
    def register(self,name):
        self.metrics[name] = copy.deepcopy(self.metric_dict)
        
    def update(self,name, label, gt):
        if name not in self.metrics.keys():
            self.register(name)
        m1 = gt != -1
        self.metrics[name]["total_valid"] += (m1).sum()
        self.metrics[name]["correct_valid"] += np.sum(gt[m1] == label[m1])
        
        m2 = (gt != -1 )*(label != -1)
        self.metrics[name]["total_valid_both"] += (m2).sum()
        self.metrics[name]["correct_valid_both"] += np.sum(gt[m2] == label[m2])
        
        self.metrics[name]["total"] += gt.size
        self.metrics[name]["correct"] += np.sum(gt == label)
        self.metrics[name]["images"] += 1
        
    def __str__(self):
        s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            acc2 = self.metrics[k]['correct_valid_both'] / self.metrics[k]['total_valid_both']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc}  Avg-Acc: valid both {acc2} \n"  
        return s
        
accm = AccMonitor()
accm.update("Test", np.ones( (100,100)) ,np.ones( (100,100)))
accm.update("Test2", np.ones( (100,100)) ,np.ones( (100,100)))
print(accm)

Acc-Monitor:
  Test: Nr-Images: 1 , Avg-Acc: 1.0  Avg-Acc: valid both 1.0 
  Test2: Nr-Images: 1 , Avg-Acc: 1.0  Avg-Acc: valid both 1.0 



In [58]:
p3

['/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt/0.png',
 '/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt/1.png',
 '/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt/2.png',
 '/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt/3.png',
 '/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt/4.png',
 '/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt/5.png',
 '/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt/6.png',
 '/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt/7.png',
 '/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels

In [64]:
plot_in_jupyter = False
accm = AccMonitor()
for k in range(len(p1)):
    gt = lla.get(p3[k*10])[0]
    network =  lla.get(p2[k])[0]
    reprojected = lla.get(p1[k])[0]
    if plot_in_jupyter:
        visu.plot_detectron(img, gt, alpha=0.8 ,store=False,jupyter=True)
        visu.plot_detectron(img, network, alpha=0.8 ,store=False,jupyter=True)
        visu.plot_detectron(img, reprojected, alpha=0.8 ,store=False,jupyter=True)
    accm.update("Reprojected", reprojected-1 , gt-1)
    accm.update("Network", network-1,gt-1)
    
    print( accm )

Acc-Monitor:
  Reprojected: Nr-Images: 1 , Avg-Acc: 0.3860462239266921  Avg-Acc: valid both 0.3860462239266921 
  Network: Nr-Images: 1 , Avg-Acc: 0.31066879332664704  Avg-Acc: valid both 0.31066879332664704 

Acc-Monitor:
  Reprojected: Nr-Images: 2 , Avg-Acc: 0.41866674134039933  Avg-Acc: valid both 0.41866674134039933 
  Network: Nr-Images: 2 , Avg-Acc: 0.33125209472946376  Avg-Acc: valid both 0.33125209472946376 

Acc-Monitor:
  Reprojected: Nr-Images: 3 , Avg-Acc: 0.4765511978875781  Avg-Acc: valid both 0.4765511978875781 
  Network: Nr-Images: 3 , Avg-Acc: 0.458571688195291  Avg-Acc: valid both 0.458571688195291 

Acc-Monitor:
  Reprojected: Nr-Images: 4 , Avg-Acc: 0.5205197374644767  Avg-Acc: valid both 0.5205197374644767 
  Network: Nr-Images: 4 , Avg-Acc: 0.5247613478809712  Avg-Acc: valid both 0.5247613478809712 

Acc-Monitor:
  Reprojected: Nr-Images: 5 , Avg-Acc: 0.5496901664407463  Avg-Acc: valid both 0.5496901664407463 
  Network: Nr-Images: 5 , Avg-Acc: 0.571247161326055

Acc-Monitor:
  Reprojected: Nr-Images: 40 , Avg-Acc: 0.46729463514499436  Avg-Acc: valid both 0.46729463514499436 
  Network: Nr-Images: 40 , Avg-Acc: 0.4280381783887632  Avg-Acc: valid both 0.4280381783887632 

Acc-Monitor:
  Reprojected: Nr-Images: 41 , Avg-Acc: 0.4653765816465604  Avg-Acc: valid both 0.4653765816465604 
  Network: Nr-Images: 41 , Avg-Acc: 0.42503735342733806  Avg-Acc: valid both 0.42503735342733806 

Acc-Monitor:
  Reprojected: Nr-Images: 42 , Avg-Acc: 0.4633822470181959  Avg-Acc: valid both 0.4633822470181959 
  Network: Nr-Images: 42 , Avg-Acc: 0.42524062888908326  Avg-Acc: valid both 0.42524062888908326 

Acc-Monitor:
  Reprojected: Nr-Images: 43 , Avg-Acc: 0.4605270489921309  Avg-Acc: valid both 0.4605270489921309 
  Network: Nr-Images: 43 , Avg-Acc: 0.425024969962768  Avg-Acc: valid both 0.425024969962768 

Acc-Monitor:
  Reprojected: Nr-Images: 44 , Avg-Acc: 0.4568069733061616  Avg-Acc: valid both 0.4568069733061616 
  Network: Nr-Images: 44 , Avg-Acc: 0.42304

Acc-Monitor:
  Reprojected: Nr-Images: 79 , Avg-Acc: 0.49938510080138776  Avg-Acc: valid both 0.49938510080138776 
  Network: Nr-Images: 79 , Avg-Acc: 0.4573508455526856  Avg-Acc: valid both 0.4573508455526856 

Acc-Monitor:
  Reprojected: Nr-Images: 80 , Avg-Acc: 0.5036524682554758  Avg-Acc: valid both 0.5036524682554758 
  Network: Nr-Images: 80 , Avg-Acc: 0.46087209870260865  Avg-Acc: valid both 0.46087209870260865 

Acc-Monitor:
  Reprojected: Nr-Images: 81 , Avg-Acc: 0.5080349933388252  Avg-Acc: valid both 0.5080349933388252 
  Network: Nr-Images: 81 , Avg-Acc: 0.46438980392090107  Avg-Acc: valid both 0.46438980392090107 

Acc-Monitor:
  Reprojected: Nr-Images: 82 , Avg-Acc: 0.5120180198645449  Avg-Acc: valid both 0.5120180198645449 
  Network: Nr-Images: 82 , Avg-Acc: 0.46761365641745056  Avg-Acc: valid both 0.46761365641745056 

Acc-Monitor:
  Reprojected: Nr-Images: 83 , Avg-Acc: 0.5156679432513739  Avg-Acc: valid both 0.5156679432513739 
  Network: Nr-Images: 83 , Avg-Acc: 0.4

Acc-Monitor:
  Reprojected: Nr-Images: 118 , Avg-Acc: 0.5916643961354557  Avg-Acc: valid both 0.5916643961354557 
  Network: Nr-Images: 118 , Avg-Acc: 0.5423767579217764  Avg-Acc: valid both 0.5423767579217764 

Acc-Monitor:
  Reprojected: Nr-Images: 119 , Avg-Acc: 0.5931962985667087  Avg-Acc: valid both 0.5931962985667087 
  Network: Nr-Images: 119 , Avg-Acc: 0.5445134768063439  Avg-Acc: valid both 0.5445134768063439 

Acc-Monitor:
  Reprojected: Nr-Images: 120 , Avg-Acc: 0.5942140560063746  Avg-Acc: valid both 0.5942140560063746 
  Network: Nr-Images: 120 , Avg-Acc: 0.5453512343885788  Avg-Acc: valid both 0.5453512343885788 

Acc-Monitor:
  Reprojected: Nr-Images: 121 , Avg-Acc: 0.5948796171779214  Avg-Acc: valid both 0.5948796171779214 
  Network: Nr-Images: 121 , Avg-Acc: 0.5441460232558295  Avg-Acc: valid both 0.5441460232558295 

Acc-Monitor:
  Reprojected: Nr-Images: 122 , Avg-Acc: 0.5951094220364839  Avg-Acc: valid both 0.5951094220364839 
  Network: Nr-Images: 122 , Avg-Acc: 0

Acc-Monitor:
  Reprojected: Nr-Images: 157 , Avg-Acc: 0.6129832694759075  Avg-Acc: valid both 0.6129832694759075 
  Network: Nr-Images: 157 , Avg-Acc: 0.5536411740319201  Avg-Acc: valid both 0.5536411740319201 

Acc-Monitor:
  Reprojected: Nr-Images: 158 , Avg-Acc: 0.6135376914334113  Avg-Acc: valid both 0.6135376914334113 
  Network: Nr-Images: 158 , Avg-Acc: 0.5542892578677023  Avg-Acc: valid both 0.5542892578677023 

Acc-Monitor:
  Reprojected: Nr-Images: 159 , Avg-Acc: 0.6139582629093516  Avg-Acc: valid both 0.6139582629093516 
  Network: Nr-Images: 159 , Avg-Acc: 0.5543239389365084  Avg-Acc: valid both 0.5543239389365084 

Acc-Monitor:
  Reprojected: Nr-Images: 160 , Avg-Acc: 0.6146002363572036  Avg-Acc: valid both 0.6146002363572036 
  Network: Nr-Images: 160 , Avg-Acc: 0.5545199861722997  Avg-Acc: valid both 0.5545199861722997 

Acc-Monitor:
  Reprojected: Nr-Images: 161 , Avg-Acc: 0.615011661189747  Avg-Acc: valid both 0.615011661189747 
  Network: Nr-Images: 161 , Avg-Acc: 0.5

...

Acc-Monitor:
  Reprojected: Nr-Images: 391 , Avg-Acc: 0.6332567490499103  Avg-Acc: valid both 0.6332567490499103 
  Network: Nr-Images: 391 , Avg-Acc: 0.569424497455644  Avg-Acc: valid both 0.569424497455644 

Acc-Monitor:
  Reprojected: Nr-Images: 392 , Avg-Acc: 0.6330676341614019  Avg-Acc: valid both 0.6330676341614019 
  Network: Nr-Images: 392 , Avg-Acc: 0.569244333708509  Avg-Acc: valid both 0.569244333708509 

Acc-Monitor:
  Reprojected: Nr-Images: 393 , Avg-Acc: 0.632836200281265  Avg-Acc: valid both 0.632836200281265 
  Network: Nr-Images: 393 , Avg-Acc: 0.569199641854038  Avg-Acc: valid both 0.569199641854038 

Acc-Monitor:
  Reprojected: Nr-Images: 394 , Avg-Acc: 0.6326581695331117  Avg-Acc: valid both 0.6326581695331117 
  Network: Nr-Images: 394 , Avg-Acc: 0.5689585152748834  Avg-Acc: valid both 0.5689585152748834 

Acc-Monitor:
  Reprojected: Nr-Images: 395 , Avg-Acc: 0.6324211738503589  Avg-Acc: valid both 0.6324211738503589 
  Network: Nr-Images: 395 , Avg-Acc: 0.5691444

Acc-Monitor:
  Reprojected: Nr-Images: 430 , Avg-Acc: 0.6315142569111915  Avg-Acc: valid both 0.6315142569111915 
  Network: Nr-Images: 430 , Avg-Acc: 0.5707084082467963  Avg-Acc: valid both 0.5707084082467963 

Acc-Monitor:
  Reprojected: Nr-Images: 431 , Avg-Acc: 0.6316186558246938  Avg-Acc: valid both 0.6316186558246938 
  Network: Nr-Images: 431 , Avg-Acc: 0.5708245135947528  Avg-Acc: valid both 0.5708245135947528 

Acc-Monitor:
  Reprojected: Nr-Images: 432 , Avg-Acc: 0.6316094227630187  Avg-Acc: valid both 0.6316094227630187 
  Network: Nr-Images: 432 , Avg-Acc: 0.5708251286400879  Avg-Acc: valid both 0.5708251286400879 

Acc-Monitor:
  Reprojected: Nr-Images: 433 , Avg-Acc: 0.6314535158568159  Avg-Acc: valid both 0.6314535158568159 
  Network: Nr-Images: 433 , Avg-Acc: 0.5706536817843761  Avg-Acc: valid both 0.5706536817843761 

Acc-Monitor:
  Reprojected: Nr-Images: 434 , Avg-Acc: 0.6312412754703944  Avg-Acc: valid both 0.6312412754703944 
  Network: Nr-Images: 434 , Avg-Acc: 0

Acc-Monitor:
  Reprojected: Nr-Images: 469 , Avg-Acc: 0.6343134862527443  Avg-Acc: valid both 0.6343134862527443 
  Network: Nr-Images: 469 , Avg-Acc: 0.5768609705064467  Avg-Acc: valid both 0.5768609705064467 

Acc-Monitor:
  Reprojected: Nr-Images: 470 , Avg-Acc: 0.6342891514378495  Avg-Acc: valid both 0.6342891514378495 
  Network: Nr-Images: 470 , Avg-Acc: 0.5766441870188483  Avg-Acc: valid both 0.5766441870188483 

Acc-Monitor:
  Reprojected: Nr-Images: 471 , Avg-Acc: 0.6342421011488408  Avg-Acc: valid both 0.6342421011488408 
  Network: Nr-Images: 471 , Avg-Acc: 0.5769125568064335  Avg-Acc: valid both 0.5769125568064335 

Acc-Monitor:
  Reprojected: Nr-Images: 472 , Avg-Acc: 0.6340001471722718  Avg-Acc: valid both 0.6340001471722718 
  Network: Nr-Images: 472 , Avg-Acc: 0.5769120789926055  Avg-Acc: valid both 0.5769120789926055 

Acc-Monitor:
  Reprojected: Nr-Images: 473 , Avg-Acc: 0.6337056716041309  Avg-Acc: valid both 0.6337056716041309 
  Network: Nr-Images: 473 , Avg-Acc: 0

Acc-Monitor:
  Reprojected: Nr-Images: 508 , Avg-Acc: 0.632963544941653  Avg-Acc: valid both 0.632963544941653 
  Network: Nr-Images: 508 , Avg-Acc: 0.5782454812196458  Avg-Acc: valid both 0.5782454812196458 

Acc-Monitor:
  Reprojected: Nr-Images: 509 , Avg-Acc: 0.6326847762987703  Avg-Acc: valid both 0.6326847762987703 
  Network: Nr-Images: 509 , Avg-Acc: 0.5778458634817983  Avg-Acc: valid both 0.5778458634817983 

Acc-Monitor:
  Reprojected: Nr-Images: 510 , Avg-Acc: 0.6324333480834932  Avg-Acc: valid both 0.6324333480834932 
  Network: Nr-Images: 510 , Avg-Acc: 0.5775337286043221  Avg-Acc: valid both 0.5775337286043221 

Acc-Monitor:
  Reprojected: Nr-Images: 511 , Avg-Acc: 0.6321537083040285  Avg-Acc: valid both 0.6321537083040285 
  Network: Nr-Images: 511 , Avg-Acc: 0.5771281865613138  Avg-Acc: valid both 0.5771281865613138 

Acc-Monitor:
  Reprojected: Nr-Images: 512 , Avg-Acc: 0.6318311328692624  Avg-Acc: valid both 0.6318311328692624 
  Network: Nr-Images: 512 , Avg-Acc: 0.5

Acc-Monitor:
  Reprojected: Nr-Images: 547 , Avg-Acc: 0.6292556654754955  Avg-Acc: valid both 0.6292556654754955 
  Network: Nr-Images: 547 , Avg-Acc: 0.570440411245446  Avg-Acc: valid both 0.570440411245446 

Acc-Monitor:
  Reprojected: Nr-Images: 548 , Avg-Acc: 0.6295955410035178  Avg-Acc: valid both 0.6295955410035178 
  Network: Nr-Images: 548 , Avg-Acc: 0.5708651952640392  Avg-Acc: valid both 0.5708651952640392 

Acc-Monitor:
  Reprojected: Nr-Images: 549 , Avg-Acc: 0.6299107052817926  Avg-Acc: valid both 0.6299107052817926 
  Network: Nr-Images: 549 , Avg-Acc: 0.5712605672334095  Avg-Acc: valid both 0.5712605672334095 

Acc-Monitor:
  Reprojected: Nr-Images: 550 , Avg-Acc: 0.6302068139834422  Avg-Acc: valid both 0.6302068139834422 
  Network: Nr-Images: 550 , Avg-Acc: 0.5716395252144599  Avg-Acc: valid both 0.5716395252144599 

Acc-Monitor:
  Reprojected: Nr-Images: 551 , Avg-Acc: 0.6304630689234536  Avg-Acc: valid both 0.6304630689234536 
  Network: Nr-Images: 551 , Avg-Acc: 0.5

In [23]:
me = accm.metrics

s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            acc2 = self.metrics[k]['correct_valid_both'] / self.metrics[k]['total_valid_both']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc}  Avg-Acc: valid both {acc2} \n"  
        return s

Acc-Monitor:
  Test: Nr-Images: 1 , Avg-Acc: 1.0 
  Test2: Nr-Images: 1 , Avg-Acc: 1.0 

